# 特征数据预处理-离散值处理

In [44]:
import pandas as pd
df = pd.DataFrame([
            ['green', 'M', 10.1, 'class1'], 
            ['red', 'L', 13.5, 'class2'], 
            ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'prize', 'class label']
df

,color,size,prize,class label
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## 标签处理

通常我们会把字符型的标签转换成数值型的

In [45]:
class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}

df['class label'] = df['class label'].map(class_mapping)
df

,color,size,prize,class label
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


## 特征处理

对于特征来说，我们一般可以做一个映射的字典

In [46]:
size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,prize,class label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


<br>
<br>

我们还可以做这样的转换进行编码

In [47]:
color_mapping = {
           'green': (0,0,1),
           'red': (0,1,0),
           'blue': (1,0,0)}

df['color'] = df['color'].map(color_mapping)
df

,color,size,prize,class label
0,"(0, 0, 1)",1,10.1,0
1,"(0, 1, 0)",2,13.5,1
2,"(1, 0, 0)",3,15.3,0


对于数据，我们同样可以给它反变换回去

In [48]:
inv_color_mapping = {v: k for k, v in color_mapping.items()}
inv_size_mapping = {v: k for k, v in size_mapping.items()}
inv_class_mapping = {v: k for k, v in class_mapping.items()}

df['color'] = df['color'].map(inv_color_mapping)
df['size'] = df['size'].map(inv_size_mapping)
df['class label'] = df['class label'].map(inv_class_mapping)
df

,color,size,prize,class label
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## Using scikit-learn and pandas features

### scikit LabelEncoder

In [49]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
df['class label'] = class_le.fit_transform(df['class label'])
df

,color,size,prize,class label
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


反变换回去可以用这个函数 `inverse_transform` :

In [50]:
class_le.inverse_transform(df['class label'])

array(['class1', 'class2', 'class1'], dtype=object)

### scikit  DictVectorizer

使用 `DictVectorizer`将得到特征的字典

In [51]:
df.transpose().to_dict().values()

dict_values([{'color': 'green', 'size': 'M', 'class label': 0, 'prize': 10.1}, {'color': 'red', 'size': 'L', 'class label': 1, 'prize': 13.5}, {'color': 'blue', 'size': 'XL', 'class label': 0, 'prize': 15.3}])

In [52]:
feature = df.iloc[:, :-1]
feature

,color,size,prize
0,green,M,10.1
1,red,L,13.5
2,blue,XL,15.3


对所有的数据都做了映射

In [53]:
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse=False)

X = dvec.fit_transform(feature.transpose().to_dict().values())
X

array([[  0. ,   1. ,   0. ,  10.1,   0. ,   1. ,   0. ],
       [  0. ,   0. ,   1. ,  13.5,   1. ,   0. ,   0. ],
       [  1. ,   0. ,   0. ,  15.3,   0. ,   0. ,   1. ]])

可以调用 `get_feature_names` 来返回新的列的名字，其中0和1就代表是不是这个属性.

In [54]:
pd.DataFrame(X, columns=dvec.get_feature_names())

,color=blue,color=green,color=red,prize,size=L,size=M,size=XL
0,0.0,1.0,0.0,10.1,0.0,1.0,0.0
1,0.0,0.0,1.0,13.5,1.0,0.0,0.0
2,1.0,0.0,0.0,15.3,0.0,0.0,1.0


### OneHotEncoder

OneHotEncoder 必须使用整数作为输入，所以得先预处理一下

In [55]:
color_le = LabelEncoder()
df['color'] = color_le.fit_transform(df['color'])

df

,color,size,prize,class label
0,1,M,10.1,0
1,2,L,13.5,1
2,0,XL,15.3,0


In [56]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

X = ohe.fit_transform(df[['color']].values)
X

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

### pandas get_dummies

Pandas库中同样有类似的操作，使用get_dummies也可以得到相应的特征

In [57]:
import pandas as pd
df = pd.DataFrame([
            ['green', 'M', 10.1, 'class1'], 
            ['red', 'L', 13.5, 'class2'], 
            ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'prize', 'class label']

size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}
df['size'] = df['size'].map(size_mapping)

class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}
df['class label'] = df['class label'].map(class_mapping)


df

,color,size,prize,class label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


对整个DF使用`get_dummies` 将会得到新的列:

In [58]:
pd.get_dummies(df)

,size,prize,class label,color_blue,color_green,color_red
0,1,10.1,0,0,1,0
1,2,13.5,1,0,0,1
2,3,15.3,0,1,0,0
